In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
charity_df = pd.read_csv('..\Resources\charity_data.csv')
charity_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


# Preprocessing

In [2]:
# What types of data do we have?
charity_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In this dataset we are looking at wheather or not the projects were successful. Therefore, our target variable is the IS_SUCCESSFUL column. There are a number of columns that we can drop for our analysis as well as they do not effect the outcome of the project such as EIN and NAME. That leaves everything else as a feature.

In [31]:
# drop EIN and NAME
noname_charity_df = charity_df.drop(columns=['EIN','NAME'])
noname_charity_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [32]:
# check the number of values for each column
noname_charity_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

Since we have a large number of values for application type and classification, we will see if we can bin some of those values. The rest we can easily handle using one hot encoding.

In [33]:
noname_charity_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

There is a precipitous drop off after T10. If we bin all the values after T10 we will have 9 columns which is easily handled using one hot encoding.

In [34]:
# Binning application type
binned_charity_df = noname_charity_df.copy()

app_counts = binned_charity_df['APPLICATION_TYPE'].value_counts()

replace_app = list(app_counts[app_counts < 500].index)

for app in replace_app:
    binned_charity_df['APPLICATION_TYPE'] = binned_charity_df['APPLICATION_TYPE'].replace(app,"Other")
    
binned_charity_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [35]:
binned_charity_df['CLASSIFICATION'].value_counts().head(15)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
Name: CLASSIFICATION, dtype: int64

Looking at the list above we can select a cutoff point of 700. This leaves 7 columns which is easily handled by one hot encoding.

In [36]:
# Binning classification
class_counts = binned_charity_df['CLASSIFICATION'].value_counts()

replace_class = list(class_counts[class_counts < 700].index)

for classi in replace_class:
    binned_charity_df['CLASSIFICATION'] = binned_charity_df['CLASSIFICATION'].replace(classi,"Other")
    
binned_charity_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

Before binning our data, we should convert the SPECIAL_CONSIDERATIONS column to a binary value.

In [37]:
binned_charity_df['SPECIAL_CONSIDERATIONS'] = binned_charity_df['SPECIAL_CONSIDERATIONS'].replace(to_replace=['N', 'Y'], value=[0, 1])
binned_charity_df.astype({'SPECIAL_CONSIDERATIONS':'int64'})
binned_charity_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,0,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,0,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,0,142590,1


In [39]:
binned_charity_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS     int64
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

Now we can encode our binned data.

In [40]:
# Use one hot encoding on categorical values

# Generate our categorical variable list
charity_cat = binned_charity_df.dtypes[binned_charity_df.dtypes == "object"].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(binned_charity_df[charity_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


Now we will merge our encoded data with our binned data.

In [43]:
# Merge one-hot encoded features and drop the originals
merge_charity_df = binned_charity_df.merge(encode_df,left_index=True, right_index=True)
merge_charity_df = merge_charity_df.drop(charity_cat,1)
merge_charity_df.head()

,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,0,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


Now that we've got our dataframe processed, we can split our training and testing data and scale it.

In [46]:
# Split our preprocessed data into our features and target arrays
y = merge_charity_df["IS_SUCCESSFUL"].values
X = merge_charity_df.drop(columns=["IS_SUCCESSFUL"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model

In [47]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  number_input_features*3

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
804/804 [==============================] - 1s 651us/step - loss: 0.5731 - accuracy: 0.7234
Epoch 2/50
804/804 [==============================] - 0s 597us/step - loss: 0.5570 - accuracy: 0.7303
Epoch 3/50
804/804 [==============================] - 0s 605us/step - loss: 0.5541 - accuracy: 0.7294
Epoch 4/50
804/804 [==============================] - 0s 596us/step - loss: 0.5522 - accuracy: 0.7315
Epoch 5/50
804/804 [==============================] - 0s 615us/step - loss: 0.5516 - accuracy: 0.7321
Epoch 6/50
804/804 [==============================] - 1s 648us/step - loss: 0.5504 - accuracy: 0.7305
Epoch 7/50
804/804 [==============================] - 0s 600us/step - loss: 0.5500 - accuracy: 0.7325
Epoch 8/50
804/804 [==============================] - 0s 611us/step - loss: 0.5490 - accuracy: 0.7323
Epoch 9/50
804/804 [==============================] - 1s 647us/step - loss: 0.5488 - accuracy: 0.7334
Epoch 10/50
804/804 [==============================] - 1s 627us/step - loss: 0.548

Our first attempt attempt resulted in an accuracy score of 73%. Because this is a complex dataset we will add a second hidden layer.

In [49]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = number_input_features*3
hidden_nodes_layer2 = number_input_features*2

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
804/804 [==============================] - 1s 725us/step - loss: 0.5697 - accuracy: 0.7231
Epoch 2/50
804/804 [==============================] - 1s 787us/step - loss: 0.5559 - accuracy: 0.7316
Epoch 3/50
804/804 [==============================] - 1s 725us/step - loss: 0.5531 - accuracy: 0.7316
Epoch 4/50
804/804 [==============================] - 1s 718us/step - loss: 0.5502 - accuracy: 0.7324
Epoch 5/50
804/804 [==============================] - 1s 722us/step - loss: 0.5482 - accuracy: 0.7330
Epoch 6/50
804/804 [==============================] - 1s 727us/step - loss: 0.5478 - accuracy: 0.7341
Epoch 7/50
804/804 [==============================] - 1s 725us/step - loss: 0.5474 - accuracy: 0.7346
Epoch 8/50
804/804 [==============================] - 1s 725us/step - loss: 0.5464 - accuracy: 0.7348
Epoch 9/50
804/804 [==============================] - 1s 729us/step - loss: 0.5455 - accuracy: 0.7359
Epoch 10/50
804/804 [==============================] - 1s 734us/step - loss: 0.544

This increased our accuracy slightly to 74%. Let's see what happens if we add another layer.

In [50]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = number_input_features*3
hidden_nodes_layer2 = number_input_features*2
hidden_nodes_layer3 = number_input_features

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
804/804 [==============================] - 1s 795us/step - loss: 0.5687 - accuracy: 0.7223
Epoch 2/50
804/804 [==============================] - 1s 868us/step - loss: 0.5538 - accuracy: 0.7311
Epoch 3/50
804/804 [==============================] - 1s 784us/step - loss: 0.5512 - accuracy: 0.7328
Epoch 4/50
804/804 [==============================] - 1s 786us/step - loss: 0.5498 - accuracy: 0.7327
Epoch 5/50
804/804 [==============================] - 1s 783us/step - loss: 0.5487 - accuracy: 0.7341
Epoch 6/50
804/804 [==============================] - 1s 781us/step - loss: 0.5476 - accuracy: 0.7333
Epoch 7/50
804/804 [==============================] - 1s 789us/step - loss: 0.5467 - accuracy: 0.7354
Epoch 8/50
804/804 [==============================] - 1s 789us/step - loss: 0.5464 - accuracy: 0.7346
Epoch 9/50
804/804 [==============================] - 1s 799us/step - loss: 0.5456 - accuracy: 0.7343
Epoch 10/50
804/804 [==============================] - 1s 867us/step - loss: 0.545

Again we have an accuracy of about 74%. This time lets change the activation function to sigmoid and increase the epochs to 100.

In [52]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = number_input_features*3
hidden_nodes_layer2 = number_input_features*2

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))



# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 1s 749us/step - loss: 0.5919 - accuracy: 0.7082
Epoch 2/100
804/804 [==============================] - 1s 755us/step - loss: 0.5747 - accuracy: 0.7231
Epoch 3/100
804/804 [==============================] - 1s 754us/step - loss: 0.5691 - accuracy: 0.7266
Epoch 4/100
804/804 [==============================] - 1s 805us/step - loss: 0.5631 - accuracy: 0.7271
Epoch 5/100
804/804 [==============================] - 1s 748us/step - loss: 0.5594 - accuracy: 0.7273
Epoch 6/100
804/804 [==============================] - 1s 740us/step - loss: 0.5566 - accuracy: 0.7292
Epoch 7/100
804/804 [==============================] - 1s 741us/step - loss: 0.5552 - accuracy: 0.7292
Epoch 8/100
804/804 [==============================] - 1s 741us/step - loss: 0.5536 - accuracy: 0.7298
Epoch 9/100
804/804 [==============================] - 1s 749us/step - loss: 0.5532 - accuracy: 0.7300
Epoch 10/100
804/804 [==============================] - 1s 741us/step - l

804/804 [==============================] - 1s 774us/step - loss: 0.5350 - accuracy: 0.7396
Epoch 80/100
804/804 [==============================] - 1s 754us/step - loss: 0.5347 - accuracy: 0.7405
Epoch 81/100
804/804 [==============================] - 1s 758us/step - loss: 0.5348 - accuracy: 0.7397
Epoch 82/100
804/804 [==============================] - 1s 755us/step - loss: 0.5347 - accuracy: 0.7402
Epoch 83/100
804/804 [==============================] - 1s 754us/step - loss: 0.5346 - accuracy: 0.7405
Epoch 84/100
804/804 [==============================] - 1s 770us/step - loss: 0.5344 - accuracy: 0.7399
Epoch 85/100
804/804 [==============================] - 1s 750us/step - loss: 0.5345 - accuracy: 0.7400
Epoch 86/100
804/804 [==============================] - 1s 751us/step - loss: 0.5341 - accuracy: 0.7400
Epoch 87/100
804/804 [==============================] - 1s 827us/step - loss: 0.5342 - accuracy: 0.7402
Epoch 88/100
804/804 [==============================] - 1s 759us/step - loss:

In [58]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = number_input_features*3
hidden_nodes_layer2 = number_input_features*2


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=300)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/300
804/804 [==============================] - 1s 743us/step - loss: 0.5718 - accuracy: 0.7215
Epoch 2/300
804/804 [==============================] - 1s 758us/step - loss: 0.5561 - accuracy: 0.7286
Epoch 3/300
804/804 [==============================] - 1s 748us/step - loss: 0.5516 - accuracy: 0.7309
Epoch 4/300
804/804 [==============================] - 1s 766us/step - loss: 0.5500 - accuracy: 0.7330
Epoch 5/300
804/804 [==============================] - 1s 786us/step - loss: 0.5490 - accuracy: 0.7332
Epoch 6/300
804/804 [==============================] - 1s 745us/step - loss: 0.5477 - accuracy: 0.7335
Epoch 7/300
804/804 [==============================] - 1s 741us/step - loss: 0.5472 - accuracy: 0.7335
Epoch 8/300
804/804 [==============================] - 1s 744us/step - loss: 0.5462 - accuracy: 0.7340
Epoch 9/300
804/804 [==============================] - 1s 749us/step - loss: 0.5451 - accuracy: 0.7327
Epoch 10/300
804/804 [==============================] - 1s 736us/step - l

804/804 [==============================] - 1s 733us/step - loss: 0.5346 - accuracy: 0.7414
Epoch 80/300
804/804 [==============================] - 1s 735us/step - loss: 0.5337 - accuracy: 0.7412
Epoch 81/300
804/804 [==============================] - 1s 733us/step - loss: 0.5342 - accuracy: 0.7410
Epoch 82/300
804/804 [==============================] - 1s 744us/step - loss: 0.5334 - accuracy: 0.7409
Epoch 83/300
804/804 [==============================] - 1s 805us/step - loss: 0.5342 - accuracy: 0.7416
Epoch 84/300
804/804 [==============================] - 1s 733us/step - loss: 0.5337 - accuracy: 0.7409
Epoch 85/300
804/804 [==============================] - 1s 738us/step - loss: 0.5334 - accuracy: 0.7409
Epoch 86/300
804/804 [==============================] - 1s 734us/step - loss: 0.5336 - accuracy: 0.7416
Epoch 87/300
804/804 [==============================] - 1s 739us/step - loss: 0.5334 - accuracy: 0.7407
Epoch 88/300
804/804 [==============================] - 1s 741us/step - loss:

804/804 [==============================] - 1s 733us/step - loss: 0.5312 - accuracy: 0.7423
Epoch 158/300
804/804 [==============================] - 1s 727us/step - loss: 0.5309 - accuracy: 0.7417
Epoch 159/300
804/804 [==============================] - 1s 738us/step - loss: 0.5316 - accuracy: 0.7429
Epoch 160/300
804/804 [==============================] - 1s 733us/step - loss: 0.5314 - accuracy: 0.7416
Epoch 161/300
804/804 [==============================] - 1s 732us/step - loss: 0.5312 - accuracy: 0.7421
Epoch 162/300
804/804 [==============================] - 1s 796us/step - loss: 0.5320 - accuracy: 0.7428
Epoch 163/300
804/804 [==============================] - 1s 732us/step - loss: 0.5310 - accuracy: 0.7420
Epoch 164/300
804/804 [==============================] - 1s 729us/step - loss: 0.5319 - accuracy: 0.7422
Epoch 165/300
804/804 [==============================] - 1s 729us/step - loss: 0.5305 - accuracy: 0.7418
Epoch 166/300
804/804 [==============================] - 1s 733us/ste

804/804 [==============================] - 1s 739us/step - loss: 0.5297 - accuracy: 0.7424
Epoch 236/300
804/804 [==============================] - 1s 729us/step - loss: 0.5301 - accuracy: 0.7428
Epoch 237/300
804/804 [==============================] - 1s 738us/step - loss: 0.5299 - accuracy: 0.7430
Epoch 238/300
804/804 [==============================] - 1s 729us/step - loss: 0.5337 - accuracy: 0.7427
Epoch 239/300
804/804 [==============================] - 1s 733us/step - loss: 0.5323 - accuracy: 0.7423
Epoch 240/300
804/804 [==============================] - 1s 733us/step - loss: 0.5298 - accuracy: 0.7431
Epoch 241/300
804/804 [==============================] - 1s 800us/step - loss: 0.5307 - accuracy: 0.7428
Epoch 242/300
804/804 [==============================] - 1s 738us/step - loss: 0.5300 - accuracy: 0.7432
Epoch 243/300
804/804 [==============================] - 1s 732us/step - loss: 0.5304 - accuracy: 0.7424
Epoch 244/300
804/804 [==============================] - 1s 735us/ste